In [20]:
import os
import imgaug.augmenters as iaa
import numpy as np
from PIL import Image

image_folder = r"C:\Users\LAB-502-09\Desktop\project-6-at-2023-07-13-11-59-c353f790\images"
annotation_folder = r"C:\Users\LAB-502-09\Desktop\project-6-at-2023-07-13-11-59-c353f790\labels"
output_folder = r"C:\Users\LAB-502-09\Desktop\project-6-at-2023-07-13-11-59-c353f790\images_augmented"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

target_image_count = 6000

augmentation_seq = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.Affine(rotate=(-10, 10)), 
    iaa.Multiply((0.8, 1.2))
])

image_files = os.listdir(image_folder)
annotation_files = os.listdir(annotation_folder)

assert len(image_files) == len(annotation_files)

augmentation_factor = int(np.ceil(target_image_count / len(image_files)))

for image_file, annotation_file in zip(image_files, annotation_files):
    image_path = os.path.join(image_folder, image_file)
    annotation_path = os.path.join(annotation_folder, annotation_file)

    image = Image.open(image_path)

    with open(annotation_path, 'r') as f:
        lines = f.readlines()

    annotations = []
    class_label = int(lines[0].split(' ')[0])
    vertices = [float(coord) for coord in lines[0].split(' ')[1:]]
    for line in lines[1:]:
        if line.startswith('0') or line.startswith('1'):
            annotations.append((class_label, vertices))
            class_label = int(line.split(' ')[0])
            vertices = [float(coord) for coord in line.split(' ')[1:]]
        else:
            vertices.extend([float(coord) for coord in line.split(' ')])

    annotations.append((class_label, vertices))

    for i in range(augmentation_factor):

        augmented_image = augmentation_seq(image=np.array(image))

        image_width, image_height = image.size
        augmented_height, augmented_width, _ = augmented_image.shape

        scale_x = augmented_width / image_width
        scale_y = augmented_height / image_height

        augmented_annotation = []
        for (class_label, vertices) in annotations:
            scaled_vertices = []
            for i in range(0, len(vertices), 2):
                x = vertices[i] * scale_x
                y = vertices[i + 1] * scale_y
                scaled_vertices.append(x)
                scaled_vertices.append(y)
            augmented_annotation.append((class_label, scaled_vertices))

        output_image_path = os.path.join(output_folder, f'augmented_{i}_{image_file}')
        Image.fromarray(augmented_image).save(output_image_path)

        output_annotation_path = os.path.join(output_folder, f'augmented_{i}_{annotation_file}')
        with open(output_annotation_path, 'w') as f:
            for (class_label, vertices) in augmented_annotation:
                line = f"{class_label} {' '.join(map(str, vertices))}\n"
                f.write(line)


augmented_image_count = len(os.listdir(output_folder))
print(f"Augmented dataset contains {augmented_image_count} images.")

Augmented dataset contains 324 images.


In [1]:
import splitfolders

In [2]:
splitfolders.ratio(r"D:\Sintu\Plant Village Tomato\Final Augmented", output="output",
    seed=1337, ratio=(.8, .1, .1), group_prefix=None, move=False)

Copying files: 8400 files [00:26, 313.41 files/s] 
